(cont:mm:virt-seg)=
# Segmentation

## Base and Bounds translation


% ![image](../images/pb-figures/mm/virt-mem-base-bounds.png)

```{figure} ../images/pb-figures/mm/virt-mem-base-bounds.png
---
width: 45%
name: mm:base-bound
---
Base-bound registers for translation 
```


We first looked at direct physical addressing, where no matter which
process is executing, the same address (e.g. 0x1000) refers to the same
memory location. In addition we reviewed a very simple form of address
translation, shown here in {numref}`mm:base-bound`
% [\[fig:vm:fig1\]](#fig:vm:fig1){reference-type="autoref" reference="fig:vm:fig1"}, 
where base and bounds registers are used to
relocate a section of the *virtual address space*---the addresses seen
by the program, corresponding to values in the CPU registers---to
somewhere else in the physical address space. By changing these
translations the operating system can create multiple virtual address
spaces, one per process; however there is still only one physical
address space, uniquely identifying each byte in each memory chip. In
this chapter we introduce *paged address translation*, a more complex
address translation mechanism used by most modern CPUs, and present the
32-bit Intel implementation as an example.

**Limitations of base+bound translation:** Modern hardware and operating
systems provide a very similar process address space model, but no
longer use base and bounds registers for address translation[^1] ,
despite it being simple, cheap, and quite possibly faster than alternate
methods. There are a number of reasons why base and bounds translation
is no longer used, but the fundamental reason is memory fragmentation.

Base and bounds address translation requires a contiguous memory region
for each process. If memory is allocated and de-allocated in chunks of
different sizes and at different times, then it can become *fragmented*
so that even if large amounts of memory are free, it will be divided
into smaller fragments, separated by longer-lived small allocations, as
seen in [\[fig:vm:fig2\]](#fig:vm:fig2){reference-type="autoref"
reference="fig:vm:fig2"}.

Start: 32 locations, all free\
![image](../images/pb-figures/mm/virt-mem-frag-1.png){width="80%"}\

Step 1, 2: a = alloc(10), b = alloc(1)\
![image](../images/pb-figures/mm/virt-mem-frag-2.png){width="80%"}\

Step 3, 4, 5: c = alloc(10), d = alloc(1), e = alloc(10)\
![image](../images/pb-figures/mm/virt-mem-frag-3.png){width="80%"}\

Step 6, 7, 8: free(a), free(c), free(e)\
![image](../images/pb-figures/mm/virt-mem-frag-4.png){width="80%"}

In the last line, you can see that only 2 units of memory (out of 32)
remain allocated, but the largest amount that can be allocated at one
time is 10 units. If all allocation requests are small, this might not
be a problem; however, in an operating system it is common to have one
or two very large processes (e.g., a web browser and word processing
software), and many small, long-running processes (e.g., the on-screen
battery display or wifi signal strength indicator). In this case, large
memory allocations may fail, even when there is enough total memory
free, because long-lived small allocations fragment the available
contiguous memory into smaller pieces.